In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
with open("sherlock holmes.txt" , "r" , encoding = "utf-8") as book:
    text = book.read()
text_vectorization_layer = tf.keras.layers.TextVectorization(split = "character" , standardize = "lower")
text_vectorization_layer.adapt([text])
encoded = text_vectorization_layer([text])[0]

In [3]:
encoded = encoded - 2
n_tokens = text_vectorization_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [4]:
def to_dataset_for_stateful_RNN(sequence , length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1 , shift = length , drop_remainder = True)
    ds = ds.flat_map(lambda window : window.batch(length + 1)).batch(1)
    return ds.map(lambda window : (window[: , :-1] , window[:,1:])).prefetch(1)
length = 100
stateful_train_set = to_dataset_for_stateful_RNN(encoded[:int(len(encoded) * 0.9)] , length= length)
stateful_validation_set = to_dataset_for_stateful_RNN(encoded[int(len(encoded)*0.9):int(len(encoded) * 0.95)] , length= length)
stateful_test_set = to_dataset_for_stateful_RNN(encoded[int(len(encoded) * 0.95):] , length= length)

In [5]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(input_dim=n_tokens , output_dim=n_tokens , batch_input_shape = [1 , None]),
        tf.keras.layers.GRU(units = 200 , return_sequences = True , stateful=True),
        tf.keras.layers.Dense(n_tokens , activation="softmax")
    ]
)
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.reset_states()
model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'nadam' , metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint("stateful_rnn_text_model" , monitor = "val_accuracy" , save_best_only=True)
history = model.fit(stateful_train_set , validation_data = stateful_validation_set , epochs=10 , callbacks=[ResetStatesCallback(),model_ckpt])

Epoch 1/10
   5122/Unknown - 205s 39ms/step - loss: 1.7024 - accuracy: 0.4930

INFO:tensorflow:Assets written to: stateful_rnn_text_model\assets


INFO:tensorflow:Assets written to: stateful_rnn_text_model\assets


5122/5122 [==============================] - 219s 42ms/step - loss: 1.7024 - accuracy: 0.4930 - val_loss: 1.4572 - val_accuracy: 0.5583
Epoch 2/10
5121/5122 [============================>.] - ETA: 0s - loss: 1.3469 - accuracy: 0.5875

INFO:tensorflow:Assets written to: stateful_rnn_text_model\assets


...

INFO:tensorflow:Assets written to: stateful_rnn_text_model\assets


5122/5122 [==============================] - 228s 45ms/step - loss: 1.1087 - accuracy: 0.6518 - val_loss: 1.2758 - val_accuracy: 0.6155
Epoch 10/10
5122/5122 [==============================] - ETA: 0s - loss: 1.1020 - accuracy: 0.6538

INFO:tensorflow:Assets written to: stateful_rnn_text_model\assets


INFO:tensorflow:Assets written to: stateful_rnn_text_model\assets


5122/5122 [==============================] - 243s 48ms/step - loss: 1.1020 - accuracy: 0.6538 - val_loss: 1.2706 - val_accuracy: 0.6157


In [6]:
stateful_text_model = tf.keras.Sequential(
    [
        text_vectorization_layer,
        tf.keras.layers.Lambda(lambda x: x-2),
        model
    ]
)
def next_character(text , temperature = 1):
    y_proba = stateful_text_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits , num_samples = 1)[0,0]
    return text_vectorization_layer.get_vocabulary()[char_id + 2]
def extend_text(text , n_chars = 1000, temperature = 1):
    for _ in range(n_chars):
        text += next_character(text , temperature)
    return text

print(extend_text("To be or not to b" , temperature=1))

1/1 [==============================] - 0s 138ms/step
To be or not to ble
      resudenes. as he was known now. ang, one or just boind having seiven
      brother.”

      “there on commanœ he had among the room london,” he ark,”

      “yes, i under the rather, and nothing lestmain. i don’t keen
      hundsor, i have seen here?”

      holmes smows his left boots in an eyez. the house was a awake the
      drilf with a new-pass and there is not in reparticalan crack whed
      happenexe, intemert of asisz-hat-chaple. “you would driends is this
      considiare-lay.”

      “precusure pallant who am impossion would hen summunse, mr. the
      bottom. it do it was still in the circams me? it was varly will him!
      to my seen hall-bushes of the road off sat scholmely in the facts
      old good arrest of indic importance negoted nothing
      other one without your certain. he was allot stand, and
      i don’t find him.”

      bucule seeming ever the night?”

      “that was come for

In [7]:
model.save('stateful_text_model.tf')

INFO:tensorflow:Assets written to: stateful_text_model.tf\assets


INFO:tensorflow:Assets written to: stateful_text_model.tf\assets
